# BoardGameGeek CSV file splitter

Previously I have made a BoardGameGeek fetcher script, that will pull all of the BGG items data from their API and store it as a single CSV file. However I realized that, later on, it would may be problematic to work with a single file, especially for the reason that `categories`, `mechanics`, `designers`, `artists` and `publishers` columns contain multiple values for each column that are comma separated, instead being in their own entry. Therefore this script will use the existing CSV file that I have collected with the previous script, and based on this, it will split the data into multiple CSV files that can be later imported as a separate tables in a database. For instance, `items.csv` will contain the main data for every item that was fetched, where `categories.csv`, `mechanics.csv`, `desginers.csv`, `artists.csv` and `publishers.csv` will contain the additional data for its respective type which is in turn related to a particular item from `items.csv`. With this, I beilieve it will be much easier to join, manipulate and process this data later in the process.

As a first step - defining of the imports that are required for this script:

In [1]:
# Define imports
import pandas as pd
from csv import DictWriter

Next will be the the creation of a function through which the output will be stored as a CSV file(s):

In [ ]:
# Function for saving results to CSV files
def save_to_csv(csv_file_name, csv_header, items):
    with open(csv_file_name, 'a', encoding='UTF8') as f:
        dictwriter_object = DictWriter(f, fieldnames=csv_header)
        if f.tell() == 0:
            dictwriter_object.writeheader()
        dictwriter_object.writerows(items)

And right below is the main logic for this script, where the data from the initial CSV file is getting processed and split into different CSV files:

In [ ]:
# Read the existing CSV file
df = pd.read_csv('./bgg.csv')

# Define an empty list that will contain all of the items
items_list = []

# Process every row in the DataFrame
for index, row in df.iterrows():
    # First define values for each row's columns and append these to items_list
    item = {
        "name": row['name'],
        "game_id": row['game_id'],
        "type": row['type'],
        "rating": row['rating'],
        "weight": row['weight'],
        "year_published": row['year_published'],
        "min_players": row['min_players'],
        "max_players": row['max_players'],
        "min_play_time": row['min_play_time'],
        "max_play_time": row['max_play_time'],
        "min_age": row['min_age'],
        "owned_by": row['owned_by']
    }
    items_list.append(item)
    
    # Split the string value for each of the following columns in order to create
    # a list of items out of it, based on comma delimiter. Each comma separated value will
    # be an item in the appropriate column's list. Append the results to their own adequate CSV file.
    additional_columns = ['categories', 'mechanics', 'designers', 'artists', 'publishers']
    for column in additional_columns:
        column_list = row[column].split(', ')
        col_header = ['game_id', column]
        for item in column_list:
            item_list = {"game_id": row['game_id'], column: item}
            save_to_csv(f"{column}.csv", col_header, [item_list])
    
# Define main items header and store the main items to a CSV file.   
items_header = [
    'name', 'game_id', 'type', 'rating', 'weight', 'year_published', 'min_players', 'max_players',
    'min_play_time', 'max_play_time', 'min_age', 'owned_by'
]
    
save_to_csv("items.csv", items_header, items_list)